In [1]:
# %pip install requests==2.28.1
# %pip install pandas==1.5.2
# %pip install openpyxl==2.6.3
# %pip install xlwings==0.30.12

IMPORT DE BIBLIOTECAS UTILIZADAS

In [2]:
import requests
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook

EXTRAÇÃO DOS DADOS

In [3]:
# Define a URL do arquivo Excel a ser baixado
url = "https://raw.githubusercontent.com/raizen-analytics/data-engineering-test/master/assets/vendas-combustiveis-m3.xls"

# Envia uma solicitação HTTP para a URL especificada
response = requests.get(url)

# Verifica se a resposta HTTP tem o código de status 200 (OK)
if response.status_code == 200:
    # Se o status for 200, abre um arquivo local em modo de escrita binária (wb) e escreve o conteudo
    with open("assets/vendas-combustiveis-m3.xls", "wb") as excel_file:
        excel_file.write(response.content)
    print(f"Arquivo Excel baixado com sucesso.")
else:
    # Se o status não for 200, exibe uma mensagem informando que o arquivo não pôde ser baixado
    print(f"Não foi possível baixar o arquivo. Código de status: {response.status_code}")


Arquivo Excel baixado com sucesso.


CONVERSÃO DE TIPO DE ARQUIVO

In [4]:
# Esta seção do código realiza a conversão de um arquivo do tipo XLS para XLSX
# a fim de aproveitar os recursos mais modernos das bibliotecas.
file_path_name = "assets/vendas-combustiveis-m3"
app = xw.App(visible=False)  # Inicializa o aplicativo Excel invisivelmente
workbook = app.books.open(f'{file_path_name}.xls')  # Abre o arquivo XLS
workbook.save(f'{file_path_name}.xlsx')  # Salva o arquivo convertido como XLSX

workbook.close()  # Fecha o arquivo original
app.quit()  # Fecha o aplicativo Excel

print(f"Arquivo convertido: '{file_path_name}.xlsx'")  # Exibe uma mensagem informando o nome do arquivo convertido


Arquivo convertido: 'assets/vendas-combustiveis-m3.xlsx'


LEITURA E SELEÇÃO DE SHEET

In [5]:
# Carrega o arquivo XLSX "vendas-combustiveis-m3.xlsx" utilizando a biblioteca correspondente
workbook = load_workbook("assets/vendas-combustiveis-m3.xlsx")

# Seleciona a planilha chamada "Plan1" dentro do arquivo XLSX carregado
worksheet_selected = workbook['Plan1']

FUNÇÃO PARA CONVERTER PIVOT EM DATAFRAME

In [6]:
def convert_pivot_cache_to_df(pivot_cache):
    rows = []  # Lista de linhas
    cols = {}  # Dicionário de colunas

    # Itera sobre as colunas no cache de tabela dinâmica e cria um dicionário de colunas
    for col in pivot_cache.cacheFields:
        cols[col.name] = col.name

    # Itera sobre os registros no cache de tabela dinâmica e cria listas de valores por registro
    for index in pivot_cache.records.r:
        row = []
        for value in index._fields:
            try:
                row.append(value.v)
            except AttributeError:
                row.append(None)

        rows.append(row)  # Adiciona a lista de valores à lista de linhas

    # Cria um DataFrame do pandas a partir das colunas e dados coletados
    df = pd.DataFrame(columns=cols, data=rows)

    return df

CRIAÇÃO DE DATAFRAMES

In [7]:
# Converte o cache de tabela dinâmica de óleo (oil) em um DataFrame
df_oil = convert_pivot_cache_to_df(worksheet_selected._pivots[3].cache)

# Converte o cache de tabela dinâmica de diesel em um DataFrame
df_diesel = convert_pivot_cache_to_df(worksheet_selected._pivots[1].cache)

TRANSFORMAÇÃO DE COLUNA PARA LINHAS

In [8]:
# Transforma o DataFrame df_oil e df_diesel utilizando a função `pd.melt`
# Mantém as colunas 'COMBUSTÍVEL', 'REGIÃO', 'ANO', 'ESTADO', 'UNIDADE' e 'TOTAL' como identificadores

df_oil_transformed = pd.melt(df_oil,
                             id_vars=['COMBUSTÍVEL', 'REGIÃO', 'ANO', 'ESTADO', 'UNIDADE', 'TOTAL'],
                             var_name='mes',
                             value_name='volume')


df_diesel_transformed = pd.melt(df_diesel,
                                id_vars=['COMBUSTÍVEL', 'REGIÃO', 'ANO', 'ESTADO', 'UNIDADE', 'TOTAL'],
                                var_name='mes',
                                value_name='volume')


CONVERSÃO DE COLUNAS DE MES E ANO

In [9]:
# Dicionário de mapeamento de nome do mês para mês numérico
month_mapping = {
    'Jan': '01',
    'Fev': '02',
    'Mar': '03',
    'Abr': '04',
    'Mai': '05',
    'Jun': '06',
    'Jul': '07',
    'Ago': '08',
    'Set': '09',
    'Out': '10',
    'Nov': '11',
    'Dez': '12'
}

# Dicionário de mapeamento do ano correspondente
year_mapping = {0: '2000', 
                1: '2001',
                2: '2002',
                3: '2003',
                4: '2004',
                5: '2005',
                6: '2006',
                7: '2007',
                8: '2008',
                9: '2009',
                10: '2010',
                11: '2011',
                12: '2012',
                13: '2013',
                14: '2014',
                15: '2015',
                16: '2016',
                17: '2017',
                18: '2018',
                19: '2019',
                20: '2020'}


# Mapeia os valores da coluna 'mes' e 'ANO' do DataFrame df_oil_transformed
# para seus equivalentes numéricos usando o dicionário de mapeamento month_mapping e year_mapping
df_oil_transformed['mes'] = df_oil_transformed['mes'].map(month_mapping)
df_oil_transformed['ANO'] = df_oil_transformed['ANO'].map(year_mapping)

# Mapeia os valores da coluna 'mes' e 'ANO' do DataFrame df_diesel_transformed
# para seus equivalentes numéricos usando o dicionário de mapeamento month_mapping e year_mapping
df_diesel_transformed['mes'] = df_diesel_transformed['mes'].map(month_mapping)
df_diesel_transformed['ANO'] = df_diesel_transformed['ANO'].map(year_mapping)

CRIAÇÃO DE COLUNA ANO_MES

In [10]:
# Cria uma coluna 'year_month' nos DataFrame que combina as colunas 'ANO' e 'mes'
df_oil_transformed['year_month'] = df_oil_transformed['ANO'] + '-' + df_oil_transformed['mes']
df_diesel_transformed['year_month'] = df_diesel_transformed['ANO'] + '-' + df_diesel_transformed['mes']

# Converte a coluna 'year_month' em um formato de data, representando o ano e mês
df_oil_transformed['year_month'] = pd.to_datetime(df_oil_transformed['year_month'], format="%Y-%m").dt.date
df_diesel_transformed['year_month'] = pd.to_datetime(df_diesel_transformed['year_month'], format="%Y-%m").dt.date


REMOÇÃO DE VALORES NAN

In [11]:
# Preenche valores ausentes (NaN) no DataFrame com 0
df_oil_transformed = df_oil_transformed.fillna(0)
df_diesel_transformed = df_diesel_transformed.fillna(0)


RENOMEIA COLUNAS E APLICA TIPAGEM

In [12]:
# Mapeia nomes de colunas em ambos os DataFrames
column_name_mapping = {
    "ESTADO": "uf",
    "UNIDADE": "unit",
    "COMBUSTÍVEL": "product"
}


# Renomeia colunas do DataFrame df_oil_transformed
df_oil_transformed = df_oil_transformed.rename(columns=column_name_mapping)
df_diesel_transformed = df_diesel_transformed.rename(columns=column_name_mapping)

# Renomeia colunas do DataFrame df_oil_transformed
df_oil_transformed['created_at'] = pd.to_datetime('today')
df_diesel_transformed['created_at'] = pd.to_datetime('today')

# Seleciona as colunas necessarias do DF
df_oil_transformed = df_oil_transformed[['year_month', 'uf', 'product', 'unit', 'volume', 'created_at']]
df_diesel_transformed = df_diesel_transformed[['year_month', 'uf', 'product', 'unit', 'volume', 'created_at']]

# Define os tipos de dados das colunas no DataFrame
df_oil_transformed = df_oil_transformed.astype({ 'uf': 'str', 'product': 'str', 'unit': 'str', 'volume': 'float64', 'created_at': 'datetime64[ns]'})
df_diesel_transformed = df_diesel_transformed.astype({'uf': 'str', 'product': 'str', 'unit': 'str', 'volume': 'float64', 'created_at': 'datetime64[ns]'})

EFETUA ESCRITA DO DADO EM PARQUET PARTICIONADO PELA COLUNA YEAR_MONTH

In [13]:
# Calcula a proporção da soma da coluna 'TOTAL' no DataFrame inicial
# em relação à soma da coluna 'volume' no DataFrame com transformaçoes.
# Se a proporção for maior ou igual a 0.999999, salva o DataFrame em formato Parquet

if df_diesel['TOTAL'].sum() / df_diesel_transformed['volume'].sum() >= 0.999999:
    df_diesel_transformed.to_parquet('assets/diesel_processed', partition_cols=['year_month'])

if df_oil['TOTAL'].sum() / df_oil_transformed['volume'].sum() >= 0.999999:
    df_oil_transformed.to_parquet('assets/oil_processed', partition_cols=['year_month'])
